In this notebook, I try using the multiple time step objective function. This is given by

$$ \sum_{j=1}^{m} | x^j - \tilde{x}^j|^2,$$
where the approximate time series is defined recursively by $\tilde{x}^0 = x^0$, $\tilde{x}^j=f(\tilde{x}^{j-1}) + (g^i + g^{i+1})\frac{h}{2}$ for $j>1$. Here, $f$ will be approximated using a neural network.

In [ ]:
import numpy as np
import xarray as xr
import torch
from scipy.linalg import eigvals
from torch.autograd import Variable

from lib.models.torch_models import predict, jacobian, train_euler_network

In [ ]:
import holoviews as hv
from lib.hvops import quadmesh
hv.extension('matplotlib')

invert_opts = dict(plot=dict(invert_yaxis=True, invert_axes=True))


Let's load the data.

In [ ]:
data = np.load("../data/ml/ngaqua/time_series_data.npz")


X = data['X']
G = data['G']
scale = data['scales']
w = data['w']

# # we need to grap the pressure field from a different path
p = xr.open_dataset("../data/raw/ngaqua/stat.nc").p.values
# t = dt * np.arange(X.shape[0])

and compute the mean.

In [ ]:
mu = np.apply_over_axes(np.mean, X, axes=(0,1,2)).ravel()
mu = mu[:-14]


sig = np.apply_over_axes(np.std, X, axes=(0,1,2)).ravel()
sig = sig[:-14]

x_mean = np.apply_over_axes(np.mean, X, axes=(0,1,2)).reshape((2,-1))
x_std = np.apply_over_axes(np.std, X, axes=(0,1,2)).reshape((2,-1))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

Let's make some interpolators for the forcing as well as the observed time series

# Multiple time step objective function

In [ ]:
def plot_soln(x, y):
    
    fig, axs = plt.subplots(2,2, figsize=(8,5), sharey=True)
    qt_levs = np.arange(11)*2.5


    t_levs = np.arange(12)*25 + 275
    t_im = axs[0,0].contourf(x[:,:34].T, levels=t_levs)
    axs[0,1].contourf(y[:,:34].T, levels=t_levs)
    q_im = axs[1,0].contourf(x[:,34:].T, levels=qt_levs)
    axs[1,1].contourf(y[:,34:].T, levels=qt_levs)

    plt.colorbar(t_im, ax=axs[0,:].tolist())
    plt.colorbar(q_im, ax=axs[1,:].tolist())
    
    axs[0,1].set_title("Prediction")
    axs[0,0].set_title("Truth")
    
    axs[0,0].set_ylabel('sl')
    axs[1,0].set_ylabel('qt')

def runsteps(step, x, n):
    """Perform n steps using"""
    out = np.zeros((n+1, x.shape[0]), dtype=x.dtype)
    out[0] = x
    
    for i in range(n):
        x = predict(step, x)
        out[i] = x
        
    return out
    

def runforcedsteps(step, x, g, n):
    """Perform n steps using"""
    out = np.zeros((n, x.shape[0]), dtype=x.dtype)
    out[0] = x
    
    for i in range(n-1):
        x = predict(step, x) + (g[i] + g[i+1])/2
        out[i] = x
        
    return out

We train the neural network outside of this notebook because it takes a long time to run. We use a 10 time step long window to compute the loss (i.e. $m=10$).

In [ ]:
ms_obj_stepper = torch.load("../data/ml/ngaqua/multistep_objective.torch")

In [ ]:
# grab a specific location
loc = [slice(None), 8, 0, slice(0, -14)]
x = X[loc]
g = G[loc]

In [ ]:
y = runsteps(ms_obj_stepper, x[0], x.shape[0])
plot_soln(x, y)

As we can see the unforced model appears to approach a reasonable steady state value.

Forced stepper

In [ ]:
# run the neural network forward
y = runforcedsteps(ms_obj_stepper, x[0], g*.125, n=x.shape[0])
plot_soln(x, y)